# Transformt Data with the constrain of 1 seconds , breaks and encoding for NLP

## Info before preprocesing

In [1]:
import pickle
import numpy as np
from Preprocesing import Preprocesing

In [2]:
# I manually move the data (locally) inside this folder 
with open('./ml4science_data_fake.pkl', 'rb') as fp:
    dl = pickle.load(fp)

In [3]:
print(dl.keys())
dl['sequences'][0].keys()

dict_keys(['sequences', 'index', 'available_demographics'])


dict_keys(['learner_id', 'sequence', 'begin', 'end', 'gender', 'year', 'language', 'label'])

In [4]:
for i in range(3):
    print(
        'begin {} - end {}: {}'.format
        (dl['sequences'][0]['begin'][i],
        dl['sequences'][0]['end'][i],
        dl['sequences'][0]['sequence'][i])
    )

begin 0 - end 7.889: [0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3440621047581669]
begin 7.889 - end 83.03: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999999, 0.0]
begin 83.03 - end 86.28: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14174189890531658]


## Repair FAKE data

In [5]:
for i in range(len(dl['sequences'])):
    for j in range(len(dl['sequences'][i]['sequence'])): 
        non_zero_count = np.count_nonzero(dl['sequences'][i]['sequence'][j])
        if non_zero_count != 2:
            #add randomly a one in of of the four first values of the sequence
            dl['sequences'][i]['sequence'][j][np.random.randint(0,4)] = 1

In [6]:
for i in range(len(dl['sequences'])):
    for j in range(len(dl['sequences'][i]['sequence'])): 
        non_zero_count = np.count_nonzero(dl['sequences'][i]['sequence'][j])
        if non_zero_count != 2:
            print(f"Vector at student {i}, sequence {j} has {non_zero_count} non-zero values")
            non_zero_positions = np.nonzero(dl['sequences'][i]['sequence'][j])[0]
            print(f"Positions of non-zero values: {non_zero_positions}")

# Manual check
print(dl['sequences'][20]['sequence'][1])
print(dl['sequences'][20]['sequence'][2])
print(dl['sequences'][28]['sequence'][1])
print(dl['sequences'][28]['sequence'][2])

[0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3564261315115266]
[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08771929824561517, 0.0, 0.0]
[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08745293331713845]


In [7]:
# save data in a pkl file
#with open('./ml4science_data_fake_repaired.pkl', 'wb') as fp:
#    pickle.dump(dl, fp)

## Test for create Vocabulary

In [8]:
token_dict={'[PAD]': 1, '[SEP]': 2, '[MASK]': 3}
token_dict['[SEP]']

2

In [9]:
tf = Preprocesing( num_states=4, num_actions=6, token_dict={'[PAD]': 1, '[SEP]': 2, '[MASK]': 3}, sep =15)

dl_class = tf.encode_vocabulary(dl)


In [10]:
for i in range(15): #range(len(dl['sequences'][0]['begin'])):
    print(
        'begin {} - end {}: {}'.format
        (dl_class['sequences'][0]['begin'][i],
        dl_class['sequences'][0]['end'][i],
        dl_class['sequences'][0]['sequence'][i])
    )

begin 0 - end 7.889: 27
begin 7.889 - end 83.03: 2
begin 83.03 - end 86.28: 27
begin 86.28 - end 86.33: 22
begin 87.278 - end 90.562: 22
begin 90.562 - end 99.597: 27
begin 99.597 - end 100.32: 24
begin 101.267 - end 107.071: 26
begin 108.989 - end 114.653: 22
begin 115.424 - end 115.935: 24
begin 117.488 - end 135.421: 2
begin 136.702 - end 144.927: 22
begin 144.927 - end 147.995: 27
begin 147.995 - end 149.956: 22
begin 150.243 - end 150.934: 22
